### AAI 511 Final Project - Group 6
#### Zach Artman, Victor Hugo Germano, Diego Acevedo

### Data Collection: Data is collected and provided to you.

### Data Pre-processing: Convert the musical scores into a format suitable for deep learning models. This involves converting the musical scores into MIDI files and applying data augmentation techniques.


### Feature Extraction: Extract features from the MIDI files, such as notes, chords, and tempo, using music analysis tools.


### Model Building: Develop a deep learning model using LSTM and CNN architectures to classify the musical scores according to the composer.


### Model Training: Train the deep learning model using the pre-processed and feature-extracted data.


### Model Evaluation: Evaluate the performance of the deep learning model using accuracy, precision, and recall metrics.


### Model Optimization: Optimize the deep learning model by fine-tuning hyperparameters.
